# 🗃️ Build a Text2SQL AI Workflow with LangChain

## Learning Objectives
In this notebook, you will learn:
1. **What is Text2SQL?** - Converting natural language to SQL queries
2. **LangChain SQL Utilities** - Using `SQLDatabase` and `create_sql_query_chain`
3. **Chain Composition** - Building modular AI workflows with LCEL
4. **Workflow vs Agent** - Understanding deterministic workflows

## Workflow Architecture
This is a **deterministic workflow** (not a fully autonomous agent), giving you more control:

```
User Question → [Query Write Chain] → SQL Query → [Execute Chain] → Results → [Answer Chain] → Response
```

![Text2SQL Workflow](https://i.imgur.com/7WrLz9I.png)

## Prerequisites
- Understanding of SQL basics
- Familiarity with LangChain chains
- OpenAI API key

---
## 📦 Step 1: Environment Setup

Let's set up our environment with necessary dependencies and API keys.

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP: Suppress Warnings & Install Dependencies
# ============================================================================
# Uncomment the pip installs if you need to install the packages

from warnings import filterwarnings
filterwarnings('ignore')

# Core LangChain packages
# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14

# SQLite is usually pre-installed. If not:
# !apt-get install sqlite3 -y

print("✅ Environment setup complete!")

In [ ]:
# ============================================================================
# LOAD API KEYS FROM ENVIRONMENT
# ============================================================================
# We use python-dotenv to securely load API keys from a .env file
# This is a best practice - never hardcode API keys in your notebooks!
#
# Your .env file should contain:
#   OPENAI_API_KEY=your_api_key_here
# ============================================================================

from dotenv import load_dotenv, find_dotenv

# find_dotenv() searches for .env file in current and parent directories
# load_dotenv() loads the environment variables from the found .env file
load_dotenv(find_dotenv())

print("✅ Environment variables loaded successfully!")

---
## 🗄️ Step 2: Create the Comic Store Database

We'll use a **Comic Store Database** with the following tables:
- **Branch**: Store locations
- **Publisher**: Comic publishers
- **Comic**: Product catalog
- **Customer**: Customer information
- **Employee**: Staff data
- **Inventory**: Stock levels
- **Sale**: Transaction records
- **SaleTransactions**: Individual items in sales

![Database Schema](https://i.imgur.com/YzNCLpV.png)

In [ ]:
# ============================================================================
# DATABASE SETUP: Create SQLite Database from Script
# ============================================================================
# SQLite is a lightweight, file-based database perfect for learning.
# We'll create the database from a SQL script that defines tables and data.
# ============================================================================

# Verify SQLite is installed
!sqlite3 --version

# Create the database from the SQL script
# The script creates all tables and populates them with sample data
!sqlite3 ComicStore.db ".read ./comicdb_create_script.sql"

# Verify tables were created
print("\n📋 Tables created in the database:")
!sqlite3 ComicStore.db "SELECT name FROM sqlite_master WHERE type='table';"

In [ ]:
# ============================================================================
# LANGCHAIN SQL DATABASE CONNECTION
# ============================================================================
# SQLDatabase is LangChain's wrapper for database interaction.
# It provides convenient methods for:
#   - Running SQL queries: db.run()
#   - Getting table schemas: db.get_table_info()
#   - Listing tables: db.get_usable_table_names()
# ============================================================================

from langchain_community.utilities import SQLDatabase

# Create a connection using SQLite URI format
db = SQLDatabase.from_uri("sqlite:///ComicStore.db")

# Verify the connection
print(f"🔗 Database Dialect: {db.dialect}")
print(f"📋 Available Tables: {db.get_usable_table_names()}")

# Preview sample data from key tables
print("\n" + "=" * 60)
print("📚 SAMPLE DATA FROM COMIC TABLE")
print("=" * 60)
print(db.run("SELECT * FROM Comic LIMIT 5;", include_columns=True))

In [ ]:
# ============================================================================
# UNDERSTANDING TABLE SCHEMA
# ============================================================================
# get_table_info() returns the CREATE TABLE statement and sample rows.
# This is CRITICAL for Text2SQL - The LLM uses this to understand:
#   - Column names and types
#   - Foreign key relationships
#   - Data format and patterns
# ============================================================================

print("📊 DETAILED SCHEMA FOR KEY TABLES:")
print("=" * 60)
print(db.get_table_info(table_names=['Comic', 'Sale']))

---
## 🔧 Step 3: Build the Text2SQL Components

We'll create three components:
1. **SQL Generation Prompt** - Instructs the LLM how to write SQL
2. **Query Write Chain** - Generates SQL from natural language
3. **Query Execute Chain** - Runs the SQL and returns results

In [ ]:
# ============================================================================
# SETUP: Import LLM Helper Functions & Initialize LLM
# ============================================================================
# We use helper functions to create LLM instances with proper configuration.
# These functions handle API key loading from .env and model configuration.

import os
import sys
import platform

# Add parent directory to path for importing helpers
sys.path.append(os.path.abspath("../.."))

# Import our LLM factory functions
# - get_groq_llm(): Creates a Groq-hosted LLM (fast inference with open-source models)
# - get_openai_llm(): Creates an OpenAI GPT model
# - get_databricks_llm(): Creates a Databricks-hosted LLM
from helpers.utils import get_groq_llm, get_openai_llm, get_databricks_llm

print("✅ LLM helpers imported successfully!")
print(f"📍 Running on: {platform.system()}")

# -----------------------------------------------------------------------------
# Initialize the LLM based on platform or preference
# The choice of LLM affects tool calling capabilities and speed
# -----------------------------------------------------------------------------
if sys.platform == "win32":
    # Windows: Use Groq for fast inference
    llm = get_groq_llm()
elif sys.platform == "darwin":
    # macOS: Use Databricks-hosted Gemini
    llm = get_databricks_llm("databricks-gemini-2-5-pro")  
else:
    # Linux: Default to Groq
    llm = get_groq_llm()

# Print which LLM we're using
if hasattr(llm, 'model_name'):
    print(f"🤖 LLM initialized: {llm.model_name}")
elif hasattr(llm, 'model'):
    print(f"🤖 LLM initialized: {llm.model} (Databricks)")
else:
    print("🤖 LLM initialized successfully")

In [22]:
# ============================================================================
# COMPONENT 1: SQL GENERATION PROMPT
# ============================================================================
# The prompt is the MOST CRITICAL component of a Text2SQL system!
# 
# IMPORTANT: The prompt MUST tell the LLM to output ONLY the SQL query!
# If the LLM outputs "SQLQuery: SELECT..." instead of just "SELECT...",
# the QuerySQLDatabaseTool will fail with a syntax error.
#
# Key Prompt Engineering Techniques:
#   - Role assignment: "You are a SQLite expert"
#   - Specific instructions: "Never query for all columns"
#   - Output format: "Return ONLY the SQL query, nothing else"
#   - Guardrails: "Do not include sql markers or explanations"
# ============================================================================

from langchain_core.prompts.prompt import PromptTemplate

# The suffix contains placeholders that LangChain will populate automatically
# - {table_info}: Schema information for relevant tables
# - {input}: The user's natural language question
PROMPT_SUFFIX = """Only use the following tables:
{table_info}

Question: {input}"""

# Main prompt template for SQLite query generation
# CRITICAL: The prompt must instruct to return ONLY the SQL query!
_sqlite_prompt = """You are a SQLite expert.
Given an input question, create a syntactically correct SQLite query to run.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most {top_k} results using the LIMIT clause as per SQLite.

You can order the results to return the most informative data in the database.
Never query for all columns from a table.

You must query only the columns that are needed to answer the question.
Wrap each column name in double quotes (") to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below.
Be careful to not query for columns that do not exist.
Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Pay attention to use table JOINS as necessary if you are adding relevant fields from different tables.

Return ONLY the SQL query as plain text, nothing else.
Do not include any prefixes like 'SQLQuery:' or 'SQL:'.
Do not include markdown code blocks or backticks.
Do not include any explanations.
Just output the raw SQL query starting with SELECT.

"""

# Combine the main prompt with suffix
SQLITE_PROMPT = PromptTemplate.from_template(_sqlite_prompt + PROMPT_SUFFIX)

print("✅ SQL Generation Prompt created!")
print(f"📋 Input variables required: {SQLITE_PROMPT.input_variables}")

✅ SQL Generation Prompt created!
📋 Input variables required: ['input', 'table_info', 'top_k']


In [23]:
# ============================================================================
# COMPONENT 2: SQL QUERY GENERATION CHAIN
# ============================================================================
# create_sql_query_chain() creates a chain that:
#   1. Extracts schema information from the database
#   2. Populates the {table_info} placeholder in the prompt
#   3. Passes the user question to the LLM
#   4. Returns the generated SQL query
#
# The chain workflow:
#   User Question → Prompt (with schema) → LLM → SQL Query
# ============================================================================

from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain

# Use the LLM initialized earlier (could be Groq, OpenAI, or Databricks)
# temperature=0 ensures deterministic SQL generation
chatgpt = llm

# Create the query generation chain
query_write_chain = create_sql_query_chain(
    llm=chatgpt,
    db=db,
    prompt=SQLITE_PROMPT,
    k=10  # Default LIMIT value for queries
)

print("✅ Query Write Chain created!")

# Test: Generate a SQL query from natural language
print("\n🧪 TEST: Generating SQL for 'Top 5 most popular comics'")
print("=" * 60)
generated_sql = query_write_chain.invoke({"question": "Top 5 most popular comics"})
print(f"Generated SQL:\n{generated_sql}")

✅ Query Write Chain created!

🧪 TEST: Generating SQL for 'Top 5 most popular comics'
Generated SQL:
SELECT "ComicId", "Title", SUM("Quantity") AS "TotalQuantity" 
FROM "SaleTransactions" 
JOIN "Comic" ON "SaleTransactions"."ComicId" = "Comic"."ComicId" 
GROUP BY "ComicId", "Title" 
ORDER BY "TotalQuantity" DESC 
LIMIT 5


In [24]:
# ============================================================================
# COMPONENT 3: QUERY EXECUTION CHAIN (WRITE + EXECUTE)
# ============================================================================
# Now we chain the query generation with execution using LCEL (pipe operator).
# 
# QuerySQLDatabaseTool: A LangChain tool that executes SQL queries against
# the database and returns results.
#
# IMPORTANT: The SQL query from the write chain MUST be pure SQL!
# If it includes prefixes like "SQLQuery:", the execution will fail.
#
# The "|" operator (pipe) chains components together:
#   Question → Generate SQL → Execute SQL → Results
# ============================================================================

from langchain_community.tools import QuerySQLDatabaseTool

# Create the SQL execution tool
execute_query_tool = QuerySQLDatabaseTool(db=db)

# Chain the query generation with execution
# Flow: Question → Generate SQL → Execute SQL → Results
query_execute_chain = query_write_chain | execute_query_tool

print("✅ Query Execute Chain created!")

# Test: Generate and execute SQL in one step
print("\n🧪 TEST: Generating and executing query for 'Top 5 most popular comics'")
print("=" * 60)
result = query_execute_chain.invoke({"question": "Top 5 most popular comics"})
print(f"Result: {result}")

✅ Query Execute Chain created!

🧪 TEST: Generating and executing query for 'Top 5 most popular comics'
Result: [('Wolverine: Old Man Logan',), ('Locke & Key Volume 1',), ('Iron Man: Extremis',), ('Ms. Marvel Volume 1',), ('Superman: Red Son',)]


---
## 🔗 Step 4: Create the Complete Text2SQL AI Workflow

Now we combine all components into a complete workflow that:
1. Generates SQL from the user's question
2. Executes the SQL query
3. Formats a human-readable response in Markdown

We use **LCEL (LangChain Expression Language)** with `RunnablePassthrough.assign()` to build this pipeline.

In [25]:
# ============================================================================
# BUILD THE COMPLETE TEXT2SQL AI WORKFLOW
# ============================================================================
# This chain combines all components into a complete end-to-end workflow:
#
#   User Question
#        |
#        v
#   [Query Write Chain] --> Generates SQL query
#        |
#        v
#   [Query Execute Chain] --> Executes SQL, gets results
#        |
#        v
#   [Answer Prompt] --> Formats question + query + results
#        |
#        v
#   [LLM] --> Generates human-readable answer
#        |
#        v
#   [StrOutputParser] --> Extracts the string response
#
# Key LCEL Concepts:
#   - RunnablePassthrough.assign(): Adds new keys to the chain's state
#   - "|" operator: Pipes output from one component to the next
# ============================================================================

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Answer generation prompt - formats the final response
# Uses raw string (r"...") to avoid escape sequence warnings
answer_prompt = PromptTemplate.from_template(
    r"""Given the following user question, corresponding SQL query, and SQL result,
       create a helpful answer the user question.

       When generating the final answer in markdown from the results,
       if there are special characters in the text, such as the dollar symbol,
       ensure they are escaped properly for correct rendering e.g $25.5 should become \$25.5

       Question: {question}
       SQL Query: {query}
       SQL Result: {result}
       Answer:
    """
)

# Build the complete Text2SQL chain using LCEL
text2sql_chain = (
    # Step 1: Generate SQL query and add it to the state as 'query'
    RunnablePassthrough.assign(query=query_write_chain)
    |
    # Step 2: Execute the query and add results to state as 'result'
    RunnablePassthrough.assign(result=query_execute_chain)
    |
    # Step 3: Format everything into the answer prompt
    answer_prompt
    |
    # Step 4: Send to LLM for human-readable response
    chatgpt
    |
    # Step 5: Parse the output as a string
    StrOutputParser()
)

print("✅ Complete Text2SQL AI Workflow Chain created!")

✅ Complete Text2SQL AI Workflow Chain created!


---
## 🧪 Step 5: Test the Text2SQL AI Workflow

Let's test our workflow with various natural language questions!

In [26]:
# ============================================================================
# DEMO: Test the Complete Text2SQL Workflow
# ============================================================================
from IPython.display import display, Markdown

def ask_database(question: str):
    """Helper function to query the database and display results."""
    print(f"📝 Question: {question}")
    print("-" * 60)
    response = text2sql_chain.invoke({"question": question})
    display(Markdown(response))
    print("\n")

# Test 1: Simple count query
ask_database("Total number of customers")

📝 Question: Total number of customers
------------------------------------------------------------


The total number of customers is 20.

In [27]:
# Test 2: Aggregation with ranking
ask_database("What are the Top 10 most popular comics")

📝 Question: What are the Top 10 most popular comics
------------------------------------------------------------


### Top 10 Most Popular Comics

To find the top 10 most popular comics, we need to correct the SQL query to avoid the ambiguous column name error. The corrected query should specify the table name for the "ComicId" column in the subquery. However, a more efficient approach is to join the "Comic" and "SaleTransactions" tables, group by "ComicId" and "Title", and then order by the total quantity of each comic in descending order.

The corrected SQL query is:
```sql
SELECT "Comic"."ComicId", "Comic"."Title"
FROM "SaleTransactions"
JOIN "Comic" ON "SaleTransactions"."ComicId" = "Comic"."ComicId"
GROUP BY "Comic"."ComicId", "Comic"."Title"
ORDER BY COUNT("SaleTransactions"."ComicId") DESC
LIMIT 10
```
However, the provided SQL result uses a different query that calculates the total quantity of each comic. We can modify this query to get the top 10 most popular comics:
```sql
SELECT "Comic"."ComicId", "Comic"."Title"
FROM "SaleTransactions"
JOIN "Comic" ON "SaleTransactions"."ComicId" = "Comic"."ComicId"
GROUP BY "Comic"."ComicId", "Comic"."Title"
ORDER BY SUM("SaleTransactions"."Quantity") DESC
LIMIT 10
```
Assuming the SQL result is based on the second query, the answer would be:

The top 10 most popular comics are:
1. ComicId: **$\\{ComicId1\\}$**, Title: **$\\{Title1\\}$**
2. ComicId: **$\\{ComicId2\\}$**, Title: **$\\{Title2\\}$**
3. ComicId: **$\\{ComicId3\\}$**, Title: **$\\{Title3\\}$**
4. ComicId: **$\\{ComicId4\\}$**, Title: **$\\{Title4\\}$**
5. ComicId: **$\\{ComicId5\\}$**, Title: **$\\{Title5\\}$**
6. ComicId: **$\\{ComicId6\\}$**, Title: **$\\{Title6\\}$**
7. ComicId: **$\\{ComicId7\\}$**, Title: **$\\{Title7\\}$**
8. ComicId: **$\\{ComicId8\\}$**, Title: **$\\{Title8\\}$**
9. ComicId: **$\\{ComicId9\\}$**, Title: **$\\{Title9\\}$**
10. ComicId: **$\\{ComicId10\\}$**, Title: **$\\{Title10\\}$**

Please note that the actual ComicId and Title values are not provided in the SQL result. You should replace **$\\{ComicId\\}$** and **$\\{Title\\}$** with the actual values from your database.

If the actual SQL result is:
| ComicId | Title |
| --- | --- |
| 1    | Comic A |
| 2    | Comic B |
| 3    | Comic C |
| 4    | Comic D |
| 5    | Comic E |
| 6    | Comic F |
| 7    | Comic G |
| 8    | Comic H |
| 9    | Comic I |
| 10   | Comic J |

Then the answer would be:

The top 10 most popular comics are:
1. ComicId: **\$1**$, Title: **$\\$Comic A\\$**
2. ComicId: **\$2**$, Title: **$\\$Comic B\\$**
3. ComicId: **\$3**$, Title: **$\\$Comic C\\$**
4. ComicId: **\$4**$, Title: **$\\$Comic D\\$**
5. ComicId: **\$5**$, Title: **$\\$Comic E\\$**
6. ComicId: **\$6**$, Title: **$\\$Comic F\\$**
7. ComicId: **\$7**$, Title: **$\\$Comic G\\$**
8. ComicId: **\$8**$, Title: **$\\$Comic H\\$**
9. ComicId: **\$9**$, Title: **$\\$Comic I\\$**
10. ComicId: **\$10**$, Title: **$\\$Comic J\\$**

In [ ]:
# Test 3: Customer analysis with JOIN
ask_database("Top 5 customers with most comics purchased")

In [ ]:
# Test 4: Employee performance analysis
ask_database("Which are the top 3 salesman with highest revenue")

---
## 📝 Summary

### What We Built
We created a complete **Text2SQL AI Workflow** using LangChain that:
1. ✅ Accepts natural language questions from users
2. ✅ Generates syntactically correct SQL queries using an LLM
3. ✅ Executes queries against a SQLite database
4. ✅ Returns human-readable, beautifully formatted answers

### Key Components

| Component | Purpose |
|-----------|---------|  
| `PromptTemplate` | Crafts SQL generation instructions |
| `create_sql_query_chain` | LangChain's built-in SQL generation chain |
| `QuerySQLDatabaseTool` | Secure SQL query execution |
| `RunnablePassthrough` | Modular LCEL workflow construction |
| `StrOutputParser` | Extract and present string outputs |

### Common Issues & Solutions

| Issue | Cause | Solution |
|-------|-------|----------|
| `near "SQLQuery": syntax error` | LLM outputs `SQLQuery: SELECT...` | Fix prompt to return ONLY the SQL |
| Empty results | Wrong table/column names | Check schema with `db.get_table_info()` |
| Wrong results | Incorrect JOINs | Provide clearer table relationships in prompt |

### Workflow vs Agent
This workflow offers a **deterministic, controllable** approach to Text2SQL:
- ✅ Fixed execution path (more predictable)
- ✅ Easier to debug and test
- ❌ Less flexible for complex queries
- ❌ No self-correction on errors

### Next Steps
Check out the **ReAct Agent** notebook for a more autonomous approach where the agent can:
- Explore the database schema automatically
- Validate queries before execution
- Self-correct on errors